# X3-slow: Hysteresis Loop — Slow Sweep (10 ep/step)

**Paper-A (P0009) — X Series: Cross-Domain Generality**

| Item | Value |
|------|-------|
| Dataset | SST-2 (binary sentiment) |
| Model | DistilBERT (pretrained) |
| Noise | Symmetric, η = 0.8 |
| Trusted ratio | 5% |
| λ path | 0.20 → 0.65 → 0.20 (Δλ = 0.05) |
| Epochs per step | **10** (slow sweep) |
| Seeds | 0–9 (n=10) |
| Total steps per run | 19 (10 up + 9 down) |
| Estimated time | ~8 hours (A100) |

**Purpose**: Test whether ascending and descending λ sweeps produce different outcomes  
at the same λ — the hallmark of history dependence (hysteresis).

**Protocol**: One model is initialized, then trained continuously as λ changes.  
Optimizer state and model weights carry over between steps (no reinitialization).


## 1. Setup

In [ ]:
!pip install -q transformers datasets accelerate

import os
import json
import time
import copy
import random
import warnings
import logging
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import (
    DistilBertTokenizer, DistilBertForSequenceClassification
)
from datasets import load_dataset

warnings.filterwarnings("ignore")
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)
os.environ["TRANSFORMERS_VERBOSITY"] = "error"
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu = torch.cuda.get_device_properties(0)
    print(f"GPU: {gpu.name} ({gpu.total_memory/1e9:.1f} GB)")


## 2. Configuration

In [ ]:
CONFIG = {
    "experiment": "exp_X3_slow_hysteresis",
    "series": "X",
    "dataset": "SST-2",
    "num_classes": 2,
    "model_name": "distilbert-base-uncased",
    "noise_type": "symmetric",
    "noise_rate": 0.8,
    "trusted_ratio": 0.05,

    # Hysteresis protocol
    "lambda_up": [0.20, 0.25, 0.30, 0.35, 0.40, 0.45, 0.50, 0.55, 0.60, 0.65],
    "lambda_down": [0.60, 0.55, 0.50, 0.45, 0.40, 0.35, 0.30, 0.25, 0.20],
    "epochs_per_step": 10,

    "seeds": list(range(10)),
    "batch_size": 64,
    "learning_rate": 2e-5,
    "weight_decay": 0.01,
    "warmup_ratio": 0.1,
    "max_seq_length": 128,
    "use_bf16": True,
    "max_train_samples": 16000,
    "csv_measure_every_n_steps": 25,

    "output_dir": "X3_slow_results",
}

os.makedirs(CONFIG["output_dir"], exist_ok=True)

# Full λ path
lambda_path = CONFIG["lambda_up"] + CONFIG["lambda_down"]
total_steps_per_run = len(lambda_path)
n_runs = len(CONFIG["seeds"])

n_used = min(int(67349 * 0.95), CONFIG["max_train_samples"])
batches_per_epoch = n_used // CONFIG["batch_size"]
updates_per_step = batches_per_epoch * CONFIG["epochs_per_step"]
total_updates = updates_per_step * total_steps_per_run
est_per_run = total_updates * 25 / 1000 + total_steps_per_run * 5  # 25ms/update + 5s eval/step

print(f"λ path: {lambda_path}")
print(f"Steps per run: {total_steps_per_run}")
print(f"Epochs per step: {CONFIG['epochs_per_step']}")
print(f"Updates per step: ~{updates_per_step}")
print(f"Total updates per run: ~{total_updates}")
print(f"Seeds: {n_runs}")
print(f"Est. per run: ~{est_per_run/60:.1f} min")
print(f"Est. total: ~{n_runs * est_per_run / 3600:.1f} h")


## 3. Google Drive Mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

DRIVE_BASE = "/content/drive/MyDrive/Paper-A_X_series"
DRIVE_OUTPUT = f"{DRIVE_BASE}/exp_X3_slow_hysteresis"
os.makedirs(DRIVE_OUTPUT, exist_ok=True)
LOCAL_OUTPUT = CONFIG["output_dir"]

def save_to_drive(filename, data):
    for d in [LOCAL_OUTPUT, DRIVE_OUTPUT]:
        with open(os.path.join(d, filename), "w") as f:
            json.dump(data, f, indent=2)

def save_figure_to_drive(fig, filename, dpi=300):
    for d in [LOCAL_OUTPUT, DRIVE_OUTPUT]:
        fig.savefig(os.path.join(d, filename), dpi=dpi, bbox_inches="tight", facecolor="white")

print(f"Drive: {DRIVE_OUTPUT}")


## 4. Deterministic Seeding

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)


## 5. Data Preparation

In [ ]:
print("Loading SST-2...")
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
raw_dataset = load_dataset("glue", "sst2")

train_texts = list(raw_dataset["train"]["sentence"])
train_labels = list(raw_dataset["train"]["label"])
test_texts = list(raw_dataset["validation"]["sentence"])
test_labels = list(raw_dataset["validation"]["label"])

print(f"Train: {len(train_texts)}, Test: {len(test_texts)}")

print("Tokenizing...")
train_encodings = tokenizer(train_texts, truncation=True, padding="max_length",
                            max_length=CONFIG["max_seq_length"], return_tensors="pt")
test_encodings = tokenizer(test_texts, truncation=True, padding="max_length",
                           max_length=CONFIG["max_seq_length"], return_tensors="pt")
print("Done.")


In [ ]:
class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        item = {k: v[idx] for k, v in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

test_dataset = TextDataset(test_encodings, torch.tensor(test_labels, dtype=torch.long))
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)
print(f"Test loader: {len(test_dataset)} samples")


## 6. Noise Injection

In [ ]:
def prepare_noisy_and_trusted(train_labels, noise_rate, trusted_ratio,
                              num_classes, seed, max_noisy_samples=None):
    rng = np.random.RandomState(seed + 10000)
    n = len(train_labels)
    labels = np.array(train_labels)
    indices = np.arange(n)

    trusted_indices = []
    for c in range(num_classes):
        class_idx = indices[labels == c]
        n_sel = max(1, int(len(class_idx) * trusted_ratio))
        trusted_indices.extend(rng.choice(class_idx, size=n_sel, replace=False))
    trusted_indices = np.array(sorted(trusted_indices))

    noisy_mask = np.ones(n, dtype=bool)
    noisy_mask[trusted_indices] = False
    noisy_indices = indices[noisy_mask]

    noisy_labels = labels.copy()
    n_flip = int(len(noisy_indices) * noise_rate)
    flip_idx = rng.choice(noisy_indices, size=n_flip, replace=False)
    for idx in flip_idx:
        candidates = [c for c in range(num_classes) if c != noisy_labels[idx]]
        noisy_labels[idx] = rng.choice(candidates)

    if max_noisy_samples and len(noisy_indices) > max_noisy_samples:
        noisy_indices = np.sort(rng.choice(noisy_indices, size=max_noisy_samples, replace=False))

    actual = np.mean(noisy_labels[noisy_indices] != labels[noisy_indices])
    print(f"  Trusted: {len(trusted_indices)} | Noisy: {len(noisy_indices)} (η={actual:.3f})")
    return torch.tensor(noisy_labels, dtype=torch.long), trusted_indices, noisy_indices

print("Test...")
_, ti, ni = prepare_noisy_and_trusted(train_labels, 0.8, 0.05, 2, 0, 16000)


## 7. Dual-Gradient Core

In [ ]:
def get_head_param_names(model):
    return [n for n, _ in model.named_parameters() if "classifier" in n or "pre_classifier" in n]

def extract_grads(model):
    return {n: p.grad.clone() for n, p in model.named_parameters() if p.grad is not None}

def normalize_grad_dict(grads):
    flat = torch.cat([g.flatten() for g in grads.values()])
    norm = flat.norm()
    if norm > 0:
        return {n: g / norm for n, g in grads.items()}, norm.item()
    return grads, 0.0

def cosine_sim(grads_a, grads_b, param_names=None):
    keys = [n for n in (param_names or sorted(set(grads_a) & set(grads_b)))
            if n in grads_a and n in grads_b]
    if not keys: return 0.0
    va = torch.cat([grads_a[n].flatten() for n in keys])
    vb = torch.cat([grads_b[n].flatten() for n in keys])
    return F.cosine_similarity(va.unsqueeze(0), vb.unsqueeze(0)).item()

def set_mixed_grad(model, g_struct_n, g_value_n, lam):
    for n, p in model.named_parameters():
        if n in g_struct_n and n in g_value_n:
            p.grad = (1 - lam) * g_struct_n[n] + lam * g_value_n[n]
        elif n in g_struct_n:
            p.grad = (1 - lam) * g_struct_n[n]
        elif n in g_value_n:
            p.grad = lam * g_value_n[n]

@torch.no_grad()
def evaluate(model, test_loader, device):
    model.eval()
    correct = total = 0
    total_loss = 0.0
    n = 0
    for batch in test_loader:
        ids = batch["input_ids"].to(device)
        mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        out = model(input_ids=ids, attention_mask=mask, labels=labels)
        preds = out.logits.argmax(dim=-1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        total_loss += out.loss.item()
        n += 1
    model.train()
    acc = correct / total
    return acc, 1.0 - acc, total_loss / n

print("Core functions defined.")


## 8. Hysteresis Sweep (Single Run)

In [ ]:
def run_hysteresis_sweep(seed, config):
    """
    Execute one full hysteresis sweep: λ↑ then λ↓.
    
    CRITICAL DESIGN:
    - Model initialized ONCE at the start
    - Optimizer state carries over between λ steps (NO reset)
    - At each step, train for epochs_per_step epochs at current λ
    - Record test error at end of each step
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    set_seed(seed)
    t_start = time.time()

    # --- Prepare data (fixed across the entire sweep) ---
    noisy_labels, trusted_idx, noisy_idx = prepare_noisy_and_trusted(
        train_labels, config["noise_rate"], config["trusted_ratio"],
        config["num_classes"], seed, config.get("max_train_samples")
    )

    noisy_dataset = TextDataset(
        {k: v[noisy_idx] for k, v in train_encodings.items()},
        noisy_labels[noisy_idx]
    )
    trusted_dataset = TextDataset(
        {k: v[trusted_idx] for k, v in train_encodings.items()},
        noisy_labels[trusted_idx]
    )

    struct_loader = DataLoader(noisy_dataset, batch_size=config["batch_size"],
                               shuffle=True, drop_last=True)
    value_loader = DataLoader(trusted_dataset,
                              batch_size=min(config["batch_size"], len(trusted_dataset)),
                              shuffle=True, drop_last=False)

    # --- Initialize model ONCE ---
    set_seed(seed)
    model = DistilBertForSequenceClassification.from_pretrained(
        config["model_name"], num_labels=config["num_classes"]
    ).to(device)
    model.train()
    head_params = get_head_param_names(model)

    # --- Optimizer: persists across ALL λ steps ---
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=config["learning_rate"],
        weight_decay=config["weight_decay"]
    )

    use_amp = config.get("use_bf16", False) and device.type == "cuda"
    amp_dtype = torch.bfloat16 if use_amp else torch.float32

    # --- Build full λ path ---
    lambda_path = config["lambda_up"] + config["lambda_down"]
    phase_labels = ["up"] * len(config["lambda_up"]) + ["down"] * len(config["lambda_down"])

    # --- Sweep ---
    sweep_log = []
    global_step = 0
    measure_every = config.get("csv_measure_every_n_steps", 25)

    for step_idx, (lam, phase) in enumerate(zip(lambda_path, phase_labels)):
        step_csv = []
        step_loss_s = 0.0
        step_loss_v = 0.0
        n_updates = 0

        for epoch in range(config["epochs_per_step"]):
            value_iter = iter(value_loader)

            for struct_batch in struct_loader:
                # Struct gradient
                optimizer.zero_grad()
                s_ids = struct_batch["input_ids"].to(device)
                s_mask = struct_batch["attention_mask"].to(device)
                s_labels = struct_batch["labels"].to(device)
                with torch.amp.autocast("cuda", dtype=amp_dtype, enabled=use_amp):
                    out_s = model(input_ids=s_ids, attention_mask=s_mask, labels=s_labels)
                out_s.loss.backward()
                g_struct = extract_grads(model)

                # Value gradient
                optimizer.zero_grad()
                try:
                    vb = next(value_iter)
                except StopIteration:
                    value_iter = iter(value_loader)
                    vb = next(value_iter)
                v_ids = vb["input_ids"].to(device)
                v_mask = vb["attention_mask"].to(device)
                v_labels = vb["labels"].to(device)
                with torch.amp.autocast("cuda", dtype=amp_dtype, enabled=use_amp):
                    out_v = model(input_ids=v_ids, attention_mask=v_mask, labels=v_labels)
                out_v.loss.backward()
                g_value = extract_grads(model)

                # Mix and step
                g_sn, _ = normalize_grad_dict(g_struct)
                g_vn, _ = normalize_grad_dict(g_value)
                set_mixed_grad(model, g_sn, g_vn, lam)
                optimizer.step()

                global_step += 1
                n_updates += 1
                step_loss_s += out_s.loss.item()
                step_loss_v += out_v.loss.item()

                if global_step % measure_every == 0:
                    c = cosine_sim(g_struct, g_value, head_params)
                    step_csv.append(c)

        # Evaluate at end of this λ step
        test_acc, test_error, test_loss = evaluate(model, test_loader, device)
        mean_csv = float(np.mean(step_csv)) if step_csv else 0.0

        step_record = {
            "step_idx": step_idx,
            "lambda": lam,
            "phase": phase,
            "test_acc": round(test_acc, 4),
            "test_error": round(test_error, 4),
            "test_loss": round(test_loss, 4),
            "loss_struct": round(step_loss_s / max(n_updates, 1), 4),
            "loss_value": round(step_loss_v / max(n_updates, 1), 4),
            "c_sv_head": round(mean_csv, 4),
            "n_updates": n_updates,
            "global_step": global_step,
        }
        sweep_log.append(step_record)

        print(f"    Step {step_idx+1}/{len(lambda_path)} [{phase:4s}] "
              f"λ={lam:.2f}: err={test_error:.4f} c_sv={mean_csv:.4f}")

    elapsed = time.time() - t_start

    result = {
        "experiment_id": f"X3slow-{seed:03d}",
        "experiment": config["experiment"],
        "seed": seed,
        "lambda_path": lambda_path,
        "phase_labels": phase_labels,
        "epochs_per_step": config["epochs_per_step"],
        "sweep_log": sweep_log,
        "total_updates": global_step,
        "time_seconds": round(elapsed, 1),
    }

    del model, optimizer
    torch.cuda.empty_cache()
    return result

print("Hysteresis sweep function defined.")


## 9. Main Execution

In [ ]:
all_results = []
n_runs = len(CONFIG["seeds"])
t_total = time.time()

print("=" * 70)
print(f"X3-slow: Hysteresis Loop — {n_runs} seeds, {CONFIG['epochs_per_step']} ep/step")
print("=" * 70)

for i, seed in enumerate(CONFIG["seeds"]):
    print(f"\nSeed {seed} ({i+1}/{n_runs})")
    print("-" * 50)
    result = run_hysteresis_sweep(seed, CONFIG)
    all_results.append(result)

    elapsed = time.time() - t_total
    eta = elapsed / (i+1) * (n_runs - i - 1)
    print(f"  Done: {result['time_seconds']:.0f}s [ETA: {eta/60:.0f}m]")

    save_to_drive(f"X3slow_seed{seed:02d}.json", result)

total_time = time.time() - t_total
save_to_drive("X3_slow_results.json", all_results)
save_to_drive("X3_slow_config.json", CONFIG)
print(f"\n{'='*70}")
print(f"COMPLETE: {n_runs} runs in {total_time/60:.1f} min")
print(f"{'='*70}")


## 10. Hysteresis Analysis

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats

# Load results
path = os.path.join(LOCAL_OUTPUT, "X3_slow_results.json")
if not os.path.exists(path):
    path = os.path.join(DRIVE_OUTPUT, "X3_slow_results.json")
with open(path) as f:
    results = json.load(f)

# Extract up/down curves per seed
lambda_up = CONFIG["lambda_up"]
lambda_down = CONFIG["lambda_down"]
n_up = len(lambda_up)
n_down = len(lambda_down)

up_errors = {lam: [] for lam in lambda_up}
down_errors = {lam: [] for lam in lambda_down}

for r in results:
    for rec in r["sweep_log"]:
        lam = rec["lambda"]
        if rec["phase"] == "up" and lam in up_errors:
            up_errors[lam].append(rec["test_error"])
        elif rec["phase"] == "down" and lam in down_errors:
            down_errors[lam].append(rec["test_error"])

# ============================================================
# Fig X3-A: Hysteresis curve
# ============================================================
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

ax = axes[0]
up_means = [np.mean(up_errors[l]) for l in lambda_up]
up_stds = [np.std(up_errors[l]) for l in lambda_up]
down_means = [np.mean(down_errors[l]) for l in lambda_down]
down_stds = [np.std(down_errors[l]) for l in lambda_down]

ax.errorbar(lambda_up, up_means, yerr=up_stds, fmt='o-', color="#D62728",
            capsize=4, linewidth=2.5, markersize=8, label="λ↑ (ascending)")
ax.fill_between(lambda_up, [m-s for m,s in zip(up_means,up_stds)],
                [m+s for m,s in zip(up_means,up_stds)], alpha=0.1, color="#D62728")

ax.errorbar(lambda_down, down_means, yerr=down_stds, fmt='s-', color="#1F77B4",
            capsize=4, linewidth=2.5, markersize=8, label="λ↓ (descending)")
ax.fill_between(lambda_down, [m-s for m,s in zip(down_means,down_stds)],
                [m+s for m,s in zip(down_means,down_stds)], alpha=0.1, color="#1F77B4")

# Draw arrows for direction
ax.annotate("", xy=(0.65, up_means[-1]+0.02), xytext=(0.20, up_means[0]+0.02),
            arrowprops=dict(arrowstyle="->", color="#D62728", lw=1.5))
ax.annotate("", xy=(0.20, down_means[-1]-0.02), xytext=(0.60, down_means[0]-0.02),
            arrowprops=dict(arrowstyle="->", color="#1F77B4", lw=1.5))

ax.axhline(0.5, color="gray", ls=":", alpha=0.4)
ax.set_xlabel("λ", fontsize=13)
ax.set_ylabel("Test Error", fontsize=13)
ax.set_title("(A) Hysteresis Curve: λ↑ vs λ↓", fontsize=14, fontweight="bold")
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

# ============================================================
# Fig X3-B: Gap at matched λ
# ============================================================
ax = axes[1]
shared_lambdas = sorted(set(lambda_up) & set(lambda_down))
gaps = []
gap_ps = []
for lam in shared_lambdas:
    if lam in up_errors and lam in down_errors:
        u = np.array(up_errors[lam])
        d = np.array(down_errors[lam])
        gap = np.mean(u) - np.mean(d)
        gaps.append(gap)
        t, p = stats.ttest_ind(u, d)
        gap_ps.append(p)

colors_bar = ["#D62728" if g > 0 else "#1F77B4" for g in gaps]
bars = ax.bar(range(len(shared_lambdas)), gaps, color=colors_bar, alpha=0.7, edgecolor="black")
for i, (g, p) in enumerate(zip(gaps, gap_ps)):
    sig = "***" if p < 0.001 else "**" if p < 0.01 else "*" if p < 0.05 else "ns"
    ax.text(i, g + (0.005 if g > 0 else -0.015), sig, ha="center", fontsize=10, fontweight="bold")
ax.set_xticks(range(len(shared_lambdas)))
ax.set_xticklabels([f"{l:.2f}" for l in shared_lambdas])
ax.axhline(0, color="black", lw=1)
ax.set_xlabel("λ", fontsize=13)
ax.set_ylabel("Gap (λ↑ error − λ↓ error)", fontsize=13)
ax.set_title("(B) Hysteresis Gap at Matched λ", fontsize=14, fontweight="bold")
ax.grid(True, alpha=0.3, axis="y")

# ============================================================
# Fig X3-C: Individual seed traces
# ============================================================
ax = axes[2]
for r in results:
    up_trace = [(rec["lambda"], rec["test_error"]) for rec in r["sweep_log"] if rec["phase"] == "up"]
    down_trace = [(rec["lambda"], rec["test_error"]) for rec in r["sweep_log"] if rec["phase"] == "down"]
    ax.plot([x[0] for x in up_trace], [x[1] for x in up_trace],
            '-', color="#D62728", alpha=0.25, linewidth=1)
    ax.plot([x[0] for x in down_trace], [x[1] for x in down_trace],
            '-', color="#1F77B4", alpha=0.25, linewidth=1)

ax.plot(lambda_up, up_means, 'o-', color="#D62728", linewidth=3, markersize=8,
        label="λ↑ mean", zorder=5)
ax.plot(lambda_down, down_means, 's-', color="#1F77B4", linewidth=3, markersize=8,
        label="λ↓ mean", zorder=5)
ax.axhline(0.5, color="gray", ls=":", alpha=0.4)
ax.set_xlabel("λ", fontsize=13)
ax.set_ylabel("Test Error", fontsize=13)
ax.set_title("(C) Individual Seed Traces", fontsize=14, fontweight="bold")
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)

fig.suptitle("X3-slow: Hysteresis Test — SST-2 / DistilBERT / η=0.8 / 10 ep per step",
             fontsize=15, fontweight="bold", y=1.02)
plt.tight_layout()
save_figure_to_drive(fig, "X3_slow_hysteresis.png")
plt.show()

# Summary stats
print("\n" + "="*60)
print("HYSTERESIS SUMMARY")
print("="*60)
print(f"{'λ':>6} | {'↑ mean':>8} | {'↓ mean':>8} | {'gap':>8} | {'p-value':>10} | {'sig':>5}")
print("-"*55)
for i, lam in enumerate(shared_lambdas):
    u = np.mean(up_errors[lam])
    d = np.mean(down_errors[lam])
    print(f"{lam:>6.2f} | {u:>8.4f} | {d:>8.4f} | {gaps[i]:>8.4f} | {gap_ps[i]:>10.2e} | "
          f"{'***' if gap_ps[i]<0.001 else '**' if gap_ps[i]<0.01 else '*' if gap_ps[i]<0.05 else 'ns':>5}")


## 11. Download

In [ ]:
print(f"Results: {DRIVE_OUTPUT}")
for f in sorted(os.listdir(DRIVE_OUTPUT)):
    sz = os.path.getsize(os.path.join(DRIVE_OUTPUT, f))
    print(f"  {f} ({sz/1024:.1f} KB)")

import shutil
shutil.make_archive("X3_slow_results", "zip", LOCAL_OUTPUT)
try:
    from google.colab import files
    files.download("X3_fast_results.zip")
except: pass
